In [1]:
from keras.models import Model
from keras.layers.core import Dense, Reshape, Lambda
from keras.layers import Input, Embedding, concatenate, Multiply
from keras import backend as K
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras import preprocessing
from keras.regularizers import l2
import random
from keras.layers.advanced_activations import LeakyReLU
import keras
import time

Using TensorFlow backend.


In [2]:
#df = pd.read_csv('/Users/BharathiSrinivasan/Documents/GitHub/Thesis/merged_data.csv')
folder = 'C:\\Users\\Pascal\\Documents\\GitHub\\instacart-market-basket-analysis\\'

In [3]:
#df_big = pd.read_csv(folder + 'merged_data.csv')

In [4]:
#Sample orders of n customer
def data_nusers(df, n):
    unique_users = df.user_id.unique()
    i = 0
    df_nusers = pd.DataFrame()  
    for user in unique_users:
        df_nusers = df_nusers.append(df[df.user_id == user])
        i +=1
        if (i == n):
            break
    return pd.DataFrame(df_nusers)

In [77]:
df = pd.read_csv(folder + 'merged_data.csv')

In [ ]:
df_use = data_nusers(df, 1000)

In [6]:
len(df_use.product_id.unique())

10761

In [7]:
# Number of product IDs available
N_products = df_use['product_id'].nunique()
N_shoppers = df_use['user_id'].nunique()

In [8]:
EMBEDDING_COLUMNS = ["user_id", "product_id"]

In [9]:
#Helper to index columns before embeddings
def val2idx(df, cols):
    val_types = dict()
    for c in cols:
        val_types[c] = df[c].unique()

    val_to_idx = dict()
    for k, v in val_types.items():
        val_to_idx[k] = {o: i for i, o in enumerate(val_types[k])}

    for k, v in val_to_idx.items():
        df[k] = df[k].apply(lambda x: v[x]+1)

    unique_vals = dict()
    for c in cols:
        unique_vals[c] = df[c].nunique()

    return df, unique_vals

In [10]:
print(N_products,N_shoppers )

10761 297


In [11]:
df_deep, unique_vals = val2idx(df_use, EMBEDDING_COLUMNS)

In [12]:
df_deep.shape

(100086, 13)

In [68]:
def first_prod(order):
    for _,row in order.iterrows():
        if row['add_to_cart_order']==1:
            return row['product_id']

In [69]:
def next_prod(order):
    for _,row in order.iterrows():
        if row['add_to_cart_order']==2:
            return row['product_id']

In [70]:
def create_basket(order):
    order['product_id']= order['product_id'].astype(str)
    
    basket = []
    for _,row in order.iterrows():
        if row['add_to_cart_order']!=1:
            basket.append(row['product_id'])
    #basket = random.shuffle(basket)
    return basket

In [46]:
def create_basket_by_order(order,order_id_list, N_products):
    names = []
    for col in range(N_products):
        names.append('col_' + str(col))
    basket_df = pd.DataFrame(columns=names, index = order_id_list)
    
    for i in order.index:
        list_items = order.get_value(i, 'basket')
        for val in list_items:
            basket_df.loc[i,'col_'+str(val)] = 1
    basket_df.fillna(0, inplace=True)
    
    return basket_df

In [47]:
order = pd.DataFrame(df_deep.groupby(['order_id']).apply(lambda order: order['product_id'].tolist()), columns=['basket'])

In [64]:
N_products

10761

In [51]:
from multiprocessing.dummy import Pool as ThreadPool 
pool = ThreadPool(4) 

In [65]:
basket_easy = df_deep.groupby(['order_id', 'product_id']).size().unstack(fill_value=0)

In [55]:
from scipy.sparse import csr_matrix

# Group interactions
order_product = df_deep.groupby(by=['order_id','product_id']).apply(lambda x: 1).to_dict() 

In [57]:
# Number of different Users / Locations
N_orders = df_deep.order_id.nunique()
N_products = df_deep.product_id.nunique()

In [61]:
# Build Rating matrix
row, col = zip(*(order_product.keys())) #row-> orders,  col-> products
map_o = dict(zip(df_deep['order_id'].unique(),range(N_orders)))
map_p = dict(zip(df_deep['product_id'].unique(),range(N_products)))
row_idx = [map_o[o] for o in row]
col_idx = [map_p[p] for p in col]
data = np.array(order_product.values)
basket_matrix = csr_matrix((data, (row_idx, col_idx)), shape=(N_orders,N_products))

TypeError: len() of unsized object

In [75]:
def transform_data_for_embedding(df):
    first = df.groupby(['order_id']).apply(first_prod)
    next_product = df.groupby(['order_id']).apply(lambda x:next_prod(x))
    basket =df.groupby(['order_id', 'product_id']).size().unstack(fill_value=0)
    transform_df = pd.DataFrame(first, columns = ['first_prod'])
    transform_df['next_product']= next_product.values
    transform_df.reset_index(inplace=True)

    # Number of product IDs available
    N_products = df['product_id'].nunique()
    N_shoppers = df['user_id'].nunique()

    return transform_df, basket, N_products, N_shoppers

In [76]:
%time
df1, basket, N_products, N_shoppers = transform_data_for_embedding(df_deep)

Wall time: 0 ns


In [54]:
#for _,row in final_df.iterrows():
#    row.basket = random.shuffle(row.basket)
    

In [20]:
def create_input_for_embed_network(df, transform_df, N_products):

    # Creating df with order_id, user_id, first prod, next prod, basket 
    x = df.drop_duplicates(subset=['order_id','user_id'])
    train_df = pd.merge(transform_df, x[['order_id','user_id']], how='left', on='order_id' )
    train_df.dropna(inplace=True)

    # Creating basket as categorical matrix for deep neural network output
    names = []
    for col in range(N_products):
        names.append('col_' + str(col)) 

    basket_df = pd.DataFrame(columns= names)
    for i,row in train_df.iterrows():
        for val in row.basket:
            if val!=0:
                basket_df.loc[i,'col_'+val] = 1
    basket_df.fillna(0, inplace=True)
    basket_df.drop(['col_0'], axis=1, inplace=True)

    train_df['next_product'] = train_df['next_product'].astype('category', categories = df.product_id.unique())
    y_df = pd.get_dummies(train_df, columns = ['next_product'])
    y_df.drop(['user_id','order_id','first_prod','basket'], axis=1, inplace=True)
    
    train_df.drop(['order_id','next_product','basket'], axis=1, inplace=True)

    return train_df['first_prod'], train_df['user_id'], basket_df, y_df

In [21]:
time_start = time.clock()
product_in , user_in, basket_in, predicted_product = create_input_for_embed_network(df_deep, df1, N_products)
time_taken = (time.clock() - time_start)

next function    order_id  first_prod  next_product  \
0         2           1           2.0   
1         3          51          52.0   
2         4         184         185.0   
3         5         168         286.0   
4         6         431         432.0   

                                              basket  
0                           [2, 3, 4, 5, 6, 7, 8, 9]  
1                       [52, 53, 54, 55, 56, 57, 58]  
2  [185, 186, 187, 188, 189, 190, 191, 192, 193, ...  
3  [286, 287, 288, 199, 289, 290, 291, 292, 293, ...  
4                                         [432, 433]  


NameError: name 'basket_in' is not defined

In [ ]:
# Integer IDs representing 1-hot encodings
prior_in = Input(shape=(1,))
shopper_in = Input(shape=(1,))

# Dense N-hot encoding for candidate products
candidates_in = Input(shape=(N_products,))

# Embeddings
prior = Embedding(N_products+1, 10)(prior_in)
shopper = Embedding(N_shoppers+1, 10)(shopper_in)

# Reshape and merge all embeddings together
reshape = Reshape(target_shape=(10,))
combined = keras.layers.concatenate([reshape(prior), reshape(shopper)])

# Hidden layers
#hidden_1 = Dense(1024, activation='relu',W_regularizer=l2(0.02))(combined)
#hidden_2 = Dense(512, activation='relu',W_regularizer=l2(0.02))(hidden_1)
hidden_3 = Dense(100, activation='relu')(combined)
#LR1 = LeakyReLU(alpha=0.1)(hidden_3)
hidden_4 = Dense(1, activation='relu')(hidden_3)

# Final 'fan-out' into the space of future products
final = Dense(N_products, activation='relu')(hidden_4)
#LR_final = LeakyReLU(alpha=0.1)(final)

# Ensure we do not overflow when we exponentiate
final = Lambda(lambda x: x - K.max(x))(final)

# Masked soft-max using Lambda and merge-multiplication
exponentiate = Lambda(lambda x: K.exp(x))(final)
masked = keras.layers.multiply([exponentiate, candidates_in])
predicted = Lambda(lambda x: x / K.sum(x))(masked)

# Compile with categorical crossentropy and adam
mdl = Model(input=[prior_in , shopper_in, candidates_in],
            output=predicted)
mdl.compile(loss='categorical_crossentropy', 
            optimizer='adam',
            metrics=['accuracy'])

In [ ]:
mdl.summary()

In [ ]:
history = mdl.fit([product_in , user_in, basket_in], predicted_product,  batch_size=128, epochs=10, verbose=0)

In [ ]:
model_json = mdl.to_json()
with open("NN_embed_model.json", "w") as json_file:
    json_file.write(model_json)
    # serialize weights to HDF5
mdl.save_weights("NN_embed_model.h5")
print("Saved model to disk")